In [1]:
import glob
import gzip
import io
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import rpy2.robjects as ro
import sys
import shutil
import tqdm

from os import listdir
from rpy2.robjects.packages import importr

%matplotlib inline

**WARNING:** Prior to running this notebook, make sure to execute the code from 01.create_sample_dataset.ipynb from the test_data folder.

### Specify input/output directories

In [2]:
# SPECIFY LOCAL PATHS
core_path = '/data/pushkare/computational_paper/GITHUB'
out_path = os.path.join(
    core_path, '00.CM_mapping', 'PHM'
)

In [3]:
# Specify names of datasets to analyze
# PHM input/output file will be stored there
datasets = ['test_data']

In [2]:
# Create dictionary with sample IDs per dataset
vcf_samples_dict = {
    dataset: pd.read_csv(
        os.path.join(
            core_path,
            dataset,
            'LCL_genotypes_chr21_chr22_samples.txt'
        ),
        sep='\t',
        header=None
    )
    for dataset in datasets
}

### Create necessary input files for PHM

1. BED file with peak coordinates
2. Binary file of normalized counts

\+ Store common samples between ChIP-seq and genotype data to ensure proper ordering of samples across data modalities

In [5]:
for dataset in datasets:
    # Create output path
    output_path = os.path.join(
        out_path,
        dataset,
        'phm_results'
    )
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    # Read in count matrices
    
    count_mtx_path = os.path.join(
        core_path,
        dataset
    )  # CHANGE PATHS
    count_mtx_k4me1 = pd.read_csv(
        os.path.join(
            count_mtx_path,
            'H3K4me1_chr21_chr22.bed'
        ),
        sep='\t'
    )
    count_mtx_k4me1['mark'] = 'H3K4me1'

    count_mtx_k27ac = pd.read_csv(
        os.path.join(
            count_mtx_path,
            'H3K27ac_chr21_chr22.bed'
        ),
        sep='\t'
    )
    count_mtx_k27ac['mark'] = 'H3K27ac'

    all_peaks_mtx = pd.concat([count_mtx_k4me1, count_mtx_k27ac])
    all_peaks_mtx = all_peaks_mtx.sort_values(['#Chr', 'start'])
    all_peaks_mtx = all_peaks_mtx.rename(columns={'#Chr': '#chr'})
    
    all_peaks_mtx['Peak'] = np.arange(1, all_peaks_mtx.shape[0] + 1)
    
    all_peaks_mtx = all_peaks_mtx.astype({'start': int, 'end': int})
    all_peaks_mtx = all_peaks_mtx.dropna(axis='columns')
    
    chromosomes = list(
        set(all_peaks_mtx.loc[:, '#chr']) - set(['chrX', 'chrY', 'chrM', 'X', 'Y', 'M'])
    )
    if not all([isinstance(el, str) for el in chromosomes]):
        sys.exit('Chromosome IDs have different types! Standardize the format')
    
    if 'chr' in str(chromosomes[0]):
        suffix = ''
    else:
        suffix = 'chr'
    
    final_samples = sorted(list(
        set(all_peaks_mtx.columns).intersection(set(vcf_samples_dict[dataset].iloc[:, 0]))
    ))
    # Store only common samples between ChIP-seq and genotype data
    pd.DataFrame(final_samples).to_csv(
        os.path.join(
            out_path,
            dataset,
            dataset + '_LCL_chr21_chr22_samples.txt'
        ),
        sep='\t',
        index=False,
        header=False
    )
    
    first_cols = ['#chr', 'start', 'end', 'pid', 'gid', 'strand']
    last_cols = ['mark', 'Peak']
    ordered_cols = first_cols + final_samples + last_cols
    columns_for_tsv = ['Peak'] + final_samples
    
    count_matrix = all_peaks_mtx.loc[:, ordered_cols].copy()

    count_matrices_by_chromosome_dict = {
        suffix + str(chromosome): chromosome_count_matrix.sort_values('start')
        for chromosome, chromosome_count_matrix in count_matrix.groupby('#chr')
        if chromosome in chromosomes
    }

    for chromosome, chromosome_count_matrix in count_matrices_by_chromosome_dict.items():
        if (chromosome != 'chrX') and (chromosome != 'chrY') and (chromosome != 'chrM'):
            path = os.path.join(output_path, str(chromosome))
            if not os.path.exists(os.path.join(path, 'hm_output'))\
            and not os.path.exists(os.path.join(path, 'phm_output')):
                os.makedirs(os.path.join(path, 'hm_output'))
                os.makedirs(os.path.join(path, 'phm_output'))
            chromosome_count_matrix['Peak'] = np.arange(
                1, 
                chromosome_count_matrix.shape[0] + 1
            )
            chromosome_count_matrix_for_tsv = chromosome_count_matrix[columns_for_tsv]

            # Create .bed file with peak coordinates
            if type(chromosome_count_matrix['#chr'].iloc[0]) != int:
                chromosome_count_matrix['#chr'] = chromosome_count_matrix['#chr'].str.replace('chr', '')
                chromosome_count_matrix['#chr'] = chromosome_count_matrix['#chr'].astype(int)
            chromosome_count_matrix[['#chr', 'start', 'end', 'mark']].to_csv(
                os.path.join(
                    path,
                    chromosome + '_peak_coordinates.bed'),
                sep='\t',
                index=False,
                header=False)

            # Create a binary file from normalized counts
            records_array = chromosome_count_matrix[final_samples].to_records(index=False)
            records_array.tofile(
                os.path.join(
                    path,
                    'normalized_counts_' + chromosome + '.bin'))

### Make sure the columns in VCF file are orderded in the same way as in the count matrices:

In [6]:
%%bash
datasets=( "test_data" );
core_path=/data/pushkare/computational_paper/GITHUB/00.CM_mapping/PHM; ## CHANGE PATH

for dataset in ${datasets[*]}
do
    bcftools view \
        -S ${core_path}/${dataset}/${dataset}_LCL_chr21_chr22_samples.txt \ ## PATH TO FILE WITH SAMPLES
        /data/pushkare/computational_paper/GITHUB/${dataset}/LCL_genotypes_chr21_chr22.vcf.gz \  ## PATH TO VCF FILE
        > ${core_path}/${dataset}/${dataset}_LCL_chr21_chr22_sample_intersection.vcf  ## OUTPUT VCF FILE
    
    bgzip ${core_path}/${dataset}/${dataset}_LCL_chr21_chr22_sample_intersection.vcf
    tabix -p vcf ${core_path}/${dataset}/${dataset}_LCL_chr21_chr22_sample_intersection.vcf.gz
done


### Compresses and index files per chromosome, per dataset

In [7]:
%%bash
datasets=( "test_data" );
core_path=/data/pushkare/computational_paper/GITHUB/00.CM_mapping/PHM;  ## CHANGE PATH
input_vcf=${core_path}/${dataset}/${dataset}_LCL_chr21_chr22_sample_intersection.vcf.gz

for dataset in ${datasets[*]}
do
    cd ${core_path}/${dataset}/phm_results
    for chromosome_dir in */
    do  
        ## Use VCF file obtained at the previous step
        input_vcf=${core_path}/${dataset}/${dataset}_LCL_chr21_chr22_sample_intersection.vcf.gz ## CHANGE PATH
        ## Subset a chromosome from VCF file
        tabix \
            ${input_vcf} \
            ${chromosome_dir%/} \
            > ${chromosome_dir%/}/${dataset}_${chromosome_dir%/}.vcf

        awk '{gsub(/^chr/,""); print}' ${chromosome_dir%/}/${dataset}_${chromosome_dir%/}.vcf \
        > ${chromosome_dir%/}/${dataset}_${chromosome_dir%/}_no_chr.vcf
        
        ## Compress and index BED, VCF files
        for filename in ${chromosome_dir%/}/*.bed
        do
            bgzip $filename
            tabix -p bed $filename.gz
        done

        for filename in ${chromosome_dir%/}/*.vcf
        do
            bgzip $filename
            tabix -p vcf $filename.gz
        done
    done
done


## if genotypes do not contain "chr", replace lines 11-14 with
## chr_id=${chromosome_dir//"chr"/}
## tabix \
##     ${core_path}/${dataset}/${dataset}_sample_intersection.vcf.gz \
##     ${chr_id%/} \
##     > ${chromosome_dir%/}/${dataset}_${chromosome_dir%/}.vcf

In [8]:
print('DONE')

DONE
